In [1]:
import os

In [2]:
project_dir = "myproject"
project_name = "Project_od_1"
dataset_name = "dataset"
label_name = "labels.names"
output_folder = "tf_training"

num_classes = 2
num_steps = 2000
batch_size = 1

#  "resnet50",  "resnet101", "inception_resnet_v2"
NN_architecture = "resnet101"

In [3]:
project_folder = os.path.join(project_dir, project_name)
project_folder

'myproject/Project_od_1'

In [4]:
dataset_path = os.path.join(project_folder, dataset_name)
dataset_path

'myproject/Project_od_1/dataset'

In [5]:
label_path = os.path.join(dataset_path, label_name)
label_path

'myproject/Project_od_1/dataset/labels.names'

In [6]:
output_folder_path =  os.path.join(project_folder, output_folder)
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
print(output_folder_path)

myproject/Project_od_1/tf_training


## Create Label Map (pbtxt)

In [7]:
def convert_classes(classes, start=1):
    msg = ''
    for id, name in enumerate(classes, start=start):
        msg = msg + "item {\n"
        msg = msg + " id: " + str(id) + "\n"
        msg = msg + " name: '" + name + "'\n}\n\n"
    return msg[:-1]

def create_label_pbtxt(label_path, output_folder_path):
    with open(label_path, 'r') as f:
        data_list = f.read().splitlines()
    
    label_pbtxt_path = os.path.join(output_folder_path, "label_map.pbtxt")
    label_map = convert_classes(data_list)
    with open( label_pbtxt_path, "w") as f:
        f.write(label_map)
        f.close()    
    print("Create 'label_map.pbtxt' Done")

In [8]:
create_label_pbtxt(label_path, output_folder_path)

Create 'label_map.pbtxt' Done


##  Creating TFRecord

In [9]:
# importing required libraraies
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

In [10]:
# Set the folder name for the source annotated XML files and folder #to store the TFRecord Record file
# LABEL_MAP_FILE=r'myproject/Project_od_1/tf_training/label_map.pbtxt'
# TRAIN_XML_FILE=r'myproject/Project_od_1/dataset/Train'
# TRAIN_TF_RECORD_DIR=r'myproject/Project_od_1/tf_training/train.record'
# TEST_XML_FILE=r'myproject/Project_od_1/dataset/eval'
# TEST_TF_RECORD_DIR=r'myproject/Project_od_1/tf_training/test.record'

LABEL_MAP_FILE = os.path.join(output_folder_path, "label_map.pbtxt")
TRAIN_XML_FILE = os.path.join(dataset_path, "Train")
TRAIN_TF_RECORD_DIR = os.path.join(output_folder_path, "train.record")
TEST_XML_FILE = os.path.join(dataset_path, "eval")
TEST_TF_RECORD_DIR = os.path.join(output_folder_path, "test.record") 

In [11]:
TRAIN_TF_RECORD_DIR

'myproject/Project_od_1/tf_training/train.record'

In [12]:
tfrecord_train = "python myproject/generate_tfrecord.py -x {} -l {} -o {}".format(TRAIN_XML_FILE, LABEL_MAP_FILE, TRAIN_TF_RECORD_DIR)
tfrecord_test = "python myproject/generate_tfrecord.py -x {} -l {} -o {}".format(TEST_XML_FILE, LABEL_MAP_FILE, TEST_TF_RECORD_DIR)

In [14]:
os.system(tfrecord_train)
os.system(tfrecord_test)

0

## Create config

In [15]:
import argparse

import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

In [16]:
pipeline = pipeline_pb2.TrainEvalPipelineConfig()     
with tf.io.gfile.GFile('myproject/sample_config/faster_rcnn_sample.config', "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline)

In [17]:
feature_extractor_dict = {
    "resnet50" : "faster_rcnn_resnet50_keras",
    "resnet101" : "faster_rcnn_resnet101_keras",
    "inception_resnet_v2" : "faster_rcnn_inception_resnet_v2_keras"
}

feature_extractor_dict = feature_extractor_dict[NN_architecture]



In [18]:
pipeline.model.faster_rcnn.num_classes = num_classes
pipeline.model.faster_rcnn.feature_extractor.type = feature_extractor_dict

In [19]:
pipeline.train_input_reader.label_map_path = LABEL_MAP_FILE
pipeline.train_input_reader.tf_record_input_reader.input_path[0] = TRAIN_TF_RECORD_DIR

pipeline.eval_input_reader[0].label_map_path = LABEL_MAP_FILE
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[0] = TEST_TF_RECORD_DIR

pipeline.train_config.num_steps = num_steps
pipeline.train_config.batch_size = batch_size

In [20]:
config_output = os.path.join(output_folder_path, "pipeline.config")
config_output

'myproject/Project_od_1/tf_training/pipeline.config'

In [21]:
config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
with tf.io.gfile.GFile(config_output, "wb") as f:                                                                                                                                                                                                                       
        f.write(config_text)

In [22]:
MODEL_DIR = os.path.join(output_folder_path, "train")
train_script = """
python object_detection/model_main_tf2.py \
  --model_dir={} \
  --pipeline_config_path={} \
  --alsologtostderr
""".format(MODEL_DIR, config_output)

In [23]:
train_script

'\npython object_detection/model_main_tf2.py   --model_dir=myproject/Project_od_1/tf_training/train   --pipeline_config_path=myproject/Project_od_1/tf_training/pipeline.config   --alsologtostderr\n'

In [24]:
#!pip install opencv-python-headless==4.2.0.34

In [25]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [26]:
!python object_detection/model_main_tf2.py   --model_dir=myproject/Project_od_1/tf_training/train   --pipeline_config_path=myproject/Project_od_1/tf_training/pipeline.config   --alsologtostderr

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0307 13:23:32.443389 139906522142528 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0307 13:23:32.446588 139906522142528 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0307 13:23:32.446658 139906522142528 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0307 13:23:32.552999 139906522142528 deprecation.py:339] From /home/tensorflow/.local/lib/python3.6/site-packages/object_detection/model_lib_v2.py:531: StrategyBase.experimental_distribute_datasets_from_function (from tensorflow.python.distribute.distribute_lib) is deprecated and will be removed in a future version.
Instructions for updating:
rename to distribute_datasets_from_

Traceback (most recent call last):
  File "object_detection/model_main_tf2.py", line 113, in <module>
    tf.compat.v1.app.run()
  File "/home/tensorflow/.local/lib/python3.6/site-packages/tensorflow/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/home/tensorflow/.local/lib/python3.6/site-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/home/tensorflow/.local/lib/python3.6/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "object_detection/model_main_tf2.py", line 110, in main
    record_summaries=FLAGS.record_summaries)
  File "/home/tensorflow/.local/lib/python3.6/site-packages/object_detection/model_lib_v2.py", line 644, in train_loop
    loss = _dist_train_step(train_input_iter)
  File "/home/tensorflow/.local/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/ho

In [27]:
# import tensorflow as tf

# def solve_cudnn_error():
#     gpus = tf.config.experimental.list_physical_devices('GPU')
#     if gpus:
#         try:
#             # Currently, memory growth needs to be the same across GPUs
#             for gpu in gpus:
#                 tf.config.experimental.set_memory_growth(gpu, True)
#             logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#             print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#         except RuntimeError as e:
#             # Memory growth must be set before GPUs have been initialized
#             print(e)

# solve_cudnn_error()

In [28]:
# device_name = tf.test.gpu_device_name()
# print(device_name)

In [29]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [30]:
# tf.config.experimental.list_physical_devices('GPU')

In [31]:
# !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2